In [1]:
import mysql.connector

config = {
      'user': 'root',
      'password': 'root',
      'host': 'localhost',
      'port': '8081',
      'database': 'breizhibus',
      'raise_on_warnings': True,
    }

#Création du Cursor
link = mysql.connector.connect(**config)
cursor = link.cursor()

#Menu
def menu_bus():
    print('\nVoici le menu de l\'application : ')
    print('\n 0. Affichage des arrêts par ligne')
    print('\n 1. Insérer un nouveau bus dans la liste')
    print('\n 2. Supprimer un bus de la liste')
    print('\n 3. Modifier un bus')
    print('\n 4. Quitter')
    
    menu_app = str(input('\n Que souhaitez vous faire ? (Taper le numéro de la commande) : '))
    
    if menu_app == '0':
        intro_app()
        menu_bus()
    elif menu_app == '1':
        insert_bus()
        menu_bus()
    elif menu_app == '2':
        del_bus()
        menu_bus()
    elif menu_app == '3':
        update_bus()
        menu_bus()
    else :
        cursor.close()
        link.close()
        print('\nMerci de votre visite.')
        
    
def intro_app():
    #Sélection de la table Lignes de bus "lignes" et execution du programme "Présentation des lignes"
    cursor.execute("SELECT * FROM lignes")
    rows_lignes = cursor.fetchall()
    
    print('\nBienvenue à Ploukuzanagi, voici les lignes de bus à disposition dans la ville :\n')
    
    for ligne in rows_lignes:
        print(f"Ligne de bus n°{ligne[0]} - {ligne[1]}")
    
    ligne_arret = int(input('\nPour connaitre l\'itinéraire d\'une ligne, sélectionnez son numéro : '))
    id_ligne = ligne_arret

    #Sélection de la table des Arrêts en fonction des Lignes de bus "arrets_lignes"
    cursor.execute("SELECT * FROM arrets INNER JOIN arrets_lignes ON arrets.id_arrets = arrets_lignes.id_arret WHERE id_ligne=%s", (id_ligne,))
    rows_arrets = cursor.fetchall()

    print('\nVoici l\'itinéraire des bus de la ligne n° {}'.format(ligne_arret)+' :\n')
    for arret in rows_arrets :
        print(f"---> Place {arret[1]}, à l\'adresse {arret[2]}.")
    
    continue_app = str(input('\nSouhaitez-vous plus d\'informations sur le(s) bus empruntant cet itinéraire ? (O/N) : '))
    
    #Sélection de la table des bus en fonction des Lignes de bus "lignes"
    if continue_app == 'O':
        cursor.execute("SELECT * FROM bus JOIN lignes ON bus.id_ligne = lignes.id_lignes WHERE id_ligne=%s", (id_ligne,))
        rows_bus = cursor.fetchall()

        print('\nLe(s) bus suivant l\'itinéraire de cette ligne :\n')
        for bus in rows_bus :
            print(f"---> Bus {bus[1]}, immatriculation {bus[2]}, contient {bus[3]} places.")
    elif continue_app == 'N' :
        print('\nRetour au menu\n')
        return menu_bus()
    else :
        print('\nMerci de sélectionner une valeur valide.')
        return intro_app()
    
#Insertion de bus
def insert_bus():
    insert_app = str(input('\nSouhaitez-vous ajouter un bus pour la ligne? (O/N) : '))
    
    if insert_app == 'O':
        insert_busforline = str(input('\n Pour quelle ligne? (Entrez le numéro de la ligne) : '))
        line = int(insert_busforline)
        numero_bus = str(input('\n Entrez le numéro du bus : '))
        immatriculation_bus = str(input('\n Entrez l\'immatriculation du bus : '))
        place_bus = str(input('\n Entrez le nombre de place dans ce bus : '))
        
        cursor.execute("""INSERT INTO `bus`(`id_bus`, `numero`, `immatriculation`, `nombre_de_place`, `id_ligne`) 
                        VALUES (NULL, %s, %s, %s, %s)""",(numero_bus, immatriculation_bus, place_bus, line,))
        print('\nLe bus '+numero_bus+' immatriculation '+immatriculation_bus+' a bien été enregistré.')
        link.commit()
        
    elif insert_app == 'N' :
        print('\nRetour au menu\n')
        return menu_bus()
    
    else :
        print('\nMerci de sélectionner une valeur valide.')
        return insert_bus()

#Suppression de bus    
def del_bus():
    continue_app = str(input('\nSouhaitez-vous continuer ? (O/N) : '))
    if continue_app == 'O':
        del_app = str(input('\nQuel bus voulez vous supprimer de la liste ? (Insérez le numéro du bus) : '))
        cursor.execute(f"DELETE FROM `bus` WHERE `numero` = '{del_app}'")
        print('Le Bus '+del_app+' a bien été supprimé')
        link.commit()
        
    elif continue_app == 'N':
        print('\nRetour au menu\n')
        return menu_bus()
    
    else :
        print('\nMerci de sélectionner une valeur valide.')
        return del_bus()

#Modification des bus
def update_bus():
    continue_app = str(input('\nSouhaitez-vous continuer ? (O/N) : '))
    if continue_app == 'O':
        cursor.execute("SELECT * FROM bus")
        rows_bus = cursor.fetchall()

        print('\nVoici les bus disponibles :\n')
        for bus in rows_bus :
            print(f"---> Bus {bus[1]}, immatriculation {bus[2]}, ligne {bus[4]}.\n")
            
        update_app = str(input('\nQuel bus voulez vous modifier ? (Entrez le numéro du bus) : ')).upper()
        update_app2 = str(input('\nEntrez un nouveau numéro : ')).upper()
        update_imm = str(input('\nSouhaitez-vous modifier son immatriculation ? (O/N)')).upper()
        
        if update_imm == 'O':
            update_imm2 = str(input('\nEntrez une nouvelle immatriculation : ')).upper()
            cursor.execute(f"UPDATE bus SET numero = '{update_app2}', immatriculation = '{update_imm2}' WHERE numero = '{update_app}'")
            print(f"\nLe bus {update_app} a bien été modifié en bus {update_app2}, immatriculation {update_imm2}")
            link.commit()

        else :
            cursor.execute(f"UPDATE bus SET numero = '{update_app2}' WHERE numero = '{update_app}'")
            print(f"\nLe bus {update_app} a bien été modifié en bus {update_app2}")
            link.commit()
            
    elif continue_app == 'N':
        print('\nRetour au menu\n')
        return menu_bus()
    
    else :
        print('\nMerci de sélectionner une valeur valide.\n')
        return update_bus()


menu_bus()


Voici le menu de l'application : 

 0. Affichage des arrêts par ligne

 1. Insérer un nouveau bus dans la liste

 2. Supprimer un bus de la liste

 3. Modifier un bus

 4. Quitter

 Que souhaitez vous faire ? (Taper le numéro de la commande) : 0

Bienvenue à Ploukuzanagi, voici les lignes de bus à disposition dans la ville :

Ligne de bus n°1 - Rouge
Ligne de bus n°2 - Vert
Ligne de bus n°3 - Bleu
Ligne de bus n°4 - Noire

Pour connaitre l'itinéraire d'une ligne, sélectionnez son numéro : 4

Voici l'itinéraire des bus de la ligne n° 4 :

---> Place Morgana, à l'adresse 2 place de Morgana.
---> Place Guénolé, à l'adresse 6 rue Saint Guénolé.

Souhaitez-vous plus d'informations sur le(s) bus empruntant cet itinéraire ? (O/N) : O

Le(s) bus suivant l'itinéraire de cette ligne :

---> Bus BB67, immatriculation ER123JJ, contient 20 places.

Voici le menu de l'application : 

 0. Affichage des arrêts par ligne

 1. Insérer un nouveau bus dans la liste

 2. Supprimer un bus de la liste

 3. 